In [34]:
import numpy as np
import scipy.fftpack as fourier
import matplotlib
import matplotlib.pyplot as plt
import scipy.io.wavfile as waves
import pyaudio as pa 
import struct 
import statsmodels.api as sm
from scipy.signal import find_peaks

In [35]:
#Definicion de la primera frecuencia de cada nota musical y del array con los nombre de las notas.

hzInicialFa = 21.826            
hzInicialFaS = 23.125           
hzInicialSol = 24.50            
hzInicialSolS = 25.96
hzInicialLa = 27.50
hzInicialLaS = 29.14
hzInicialSi = 30.87
hzInicialDo = 32.7              
hzInicialDoS = 34.65
hzInicialRe = 36.71
hzInicialReS = 38.89
hzInicialMi = 41.2

hzIniciales = np.array([hzInicialFa, hzInicialFaS, hzInicialSol, hzInicialSolS, hzInicialLa, hzInicialLaS, hzInicialSi, hzInicialDo, hzInicialDoS, hzInicialRe, hzInicialReS, hzInicialMi], dtype=float)

nombreNota = np.array(['Fa','Fa#','Sol','Sol#','La','La#','Si','Do','Do#','Re','Re#','Mi'], dtype = str)

In [36]:
#Funciones para conseguir los 0.5 anteriores y los 0.5 seguientes de un Hz

def nAnteriores(Hz):
    vAnteriores = np.array([], dtype=float)
    for i in range(5):
        valor = 5 - i
        vAnteriores = np.append(vAnteriores, (Hz - valor))
    return vAnteriores

def nSiguientes(Hz):
    vSiguientes = np.array([], dtype=float)
    for i in range(5):
        valor = i + 1
        vSiguientes = np.append(vSiguientes, (Hz + valor))
    return vSiguientes

#Funciones para acumular los valores anteiores y siguientes.

def vAnteriores(Hz):
    valoresAnteriores = np.array([], dtype=float)
    valoresAnteriores = np.append(valoresAnteriores, nAnteriores(Hz))
    valoresAnteriores = np.append(valoresAnteriores, Hz)
    return valoresAnteriores

def vSiguientes(Hz):
    valoresSiguientes = np.array([], dtype=float)
    valoresSiguientes = np.append(valoresSiguientes, nSiguientes(Hz))
    return valoresSiguientes

#Funcion para el guardado automatico de las frecuencias en un array.

def LlenadoAutomaticoDeHz(Hz):
    allHz = np.array([], dtype=float)
    allHz = np.append(allHz, vAnteriores(Hz))
    allHz = np.append(allHz, vSiguientes(Hz))
    return allHz

In [37]:
#Frecuencias de Fa, Fa# y Sol
FaFaSSolHz = np.array([], dtype=float)
for i in range(3):                                                          #Tomo las notas Fa, Fa# y Sol
    for j in range(9):                                                      
        Hz2 = hzIniciales[i] * (2 ** j)                                     #Multiplico por 2 para que sean las notas de la octava superior
        FaFaSSolHz = np.append(FaFaSSolHz, LlenadoAutomaticoDeHz(Hz2))

#Frecuencias de Sol#, La, La#, Si, Do, Do#, Re, Re#, Mi
allHz = np.array([], dtype=float)
for i in range(3, 12):
    for j in range(8):
        Hz2 = hzIniciales[i] * (2 ** j)
        allHz = np.append(allHz, LlenadoAutomaticoDeHz(Hz2))


In [38]:
#Ingreso de las frecuencias en los array de las notas Fa, Fa#, Sol
# 99 valores en total para cada nota.

total = 99

FaHz = np.array([], dtype=float)
for i in range(total):
    FaHz = np.append(FaHz, FaFaSSolHz[i])

FaSHz = np.array([], dtype=float)
for i in range(total, total*2):
    FaSHz = np.append(FaSHz, FaFaSSolHz[i])

SolHz = np.array([], dtype=float)
for i in range(total*2, total*3):
    SolHz = np.append(SolHz, FaFaSSolHz[i])

#Ingreso de las frecuencias en los array de las notas Sol#, La, La#, Si, Do, Do#, Re, Re#, Mi
# 88 valores en total para cada nota.

total2 = 88

SolSHz = np.array([], dtype=float)
for i in range(total2):
    SolSHz = np.append(SolSHz, allHz[i])

LaHz = np.array([], dtype=float)
for i in range(total2, total2*2):
    LaHz = np.append(LaHz, allHz[i])

LaSHz = np.array([], dtype=float)
for i in range(total2*2, total2*3):
    LaSHz = np.append(LaSHz, allHz[i])

SiHz = np.array([], dtype=float)
for i in range(total2*3, total2*4):
    SiHz = np.append(SiHz, allHz[i])

DoHz = np.array([], dtype=float)
for i in range(total2*4, total2*5):
    DoHz = np.append(DoHz, allHz[i])

DoSHz = np.array([], dtype=float)
for i in range(total2*5, total2*6):
    DoSHz = np.append(DoSHz, allHz[i])

ReHz = np.array([], dtype=float)
for i in range(total2*6, total2*7):
    ReHz = np.append(ReHz, allHz[i])

ReSHz = np.array([], dtype=float)
for i in range(total2*7, total2*8):
    ReSHz = np.append(ReSHz, allHz[i])

MiHz = np.array([], dtype=float)
for i in range(total2*8, total2*9):
    MiHz = np.append(MiHz, allHz[i])

In [39]:
#.---.HACIA ARRIBA ES LA DEFINICION DE LOS HZ Y SU ALMACENAMIENTO EN SUS RESPECTIVAS NOTAS MUSICALES.---.#

In [40]:
HzMaximo = 30.87
posicionHz = np.where(allHz==HzMaximo)[0][0]
print(posicionHz)
print(len(allHz))
print(allHz[posicionHz])

269
792
30.87


In [41]:
frecuencia = 1909
contenedorDeFrecuencias = np.array([10, 12, 100, 500], dtype=float)
aproximacion = contenedorDeFrecuencias.flat[np.abs(contenedorDeFrecuencias - frecuencia).argmin()]
print(aproximacion)
diferencia = frecuencia - aproximacion
print(diferencia)
if diferencia > 10 : print(frecuencia)
elif diferencia < -10 : print(frecuencia)
else : print("es un do D:")

500.0
1409.0
1909


In [42]:
FaFaSSolHz.flat[np.abs(allHz - 440).argmin()]

185.0

In [43]:
def determinarNotaMusical(HzMaximo):

    posicionHz = np.where(FaFaSSolHz==HzMaximo)[0][0]

    if (HzMaximo in FaHz): print( "Es un Fa en: ", FaFaSSolHz[posicionHz], "Hz")
    elif (HzMaximo in FaSHz): print( "Es un Fa# en: ", FaFaSSolHz[posicionHz], "Hz")
    elif (HzMaximo in SolHz): print( "Es un Sol en: ", FaFaSSolHz[posicionHz], "Hz")

def determinarNotaMusical2(HzMaximo):

    posicionHz = np.where(allHz==HzMaximo)[0][0]

    if (HzMaximo in SolSHz): print( "Es un Sol# en: ", allHz[posicionHz], "Hz")
    elif (HzMaximo in LaHz): print( "Es un La en: ", allHz[posicionHz], "Hz")
    elif (HzMaximo in LaSHz): print( "Es un La# en: ", allHz[posicionHz], "Hz")
    elif (HzMaximo in SiHz): print( "Es un Si en: ", allHz[posicionHz], "Hz")
    elif (HzMaximo in DoHz): print( "Es un Do en: ", allHz[posicionHz], "Hz")
    elif (HzMaximo in DoSHz): print( "Es un Do# en: ", allHz[posicionHz], "Hz")
    elif (HzMaximo in ReHz): print( "Es un Re en: ", allHz[posicionHz], "Hz")
    elif (HzMaximo in ReSHz): print( "Es un Re# en: ", allHz[posicionHz], "Hz")
    elif (HzMaximo in MiHz): print( "Es un Mi en: ", allHz[posicionHz], "Hz")

In [44]:
def aproximar (Frecuencias, HzMaximo):
    aproximacion = Frecuencias.flat[np.abs(Frecuencias - HzMaximo).argmin()]
    return aproximacion

In [45]:
#4698.88
f = 4695.88
f2 = 4700.88
aprox = aproximar(FaFaSSolHz, f)
aprox2 = aproximar(allHz, f2)
print(aprox)
print(aprox2)

diferencia = f - aprox
diferencia2 = f2 - aprox2
print("d1: ", diferencia)
print("d2: ", diferencia2)

if diferencia < 5 and diferencia > -5:
    if diferencia2 < 5 and diferencia2 > -5:
        if diferencia < diferencia2:
            print("toy en el 1er IF")
            determinarNotaMusical(aprox)
        elif diferencia > diferencia2:
            print("toy en el 2do IF")
            determinarNotaMusical2(aprox2)
        else:
            print("toy en el 3er IF")
            determinarNotaMusical(aprox)
            determinarNotaMusical2(aprox2)
    else: determinarNotaMusical(aprox)
elif diferencia2 < 5 and diferencia2 > -5:
    determinarNotaMusical2(aprox2)
else:
    print(aprox, "Hz")
    print(aprox2, "Hz")



if diferencia < diferencia2 :
    if diferencia > 1.5 : 
        print(aprox, "Hz")
    elif diferencia < -2 : 
        print(aprox, "Hz")
    else : determinarNotaMusical(aprox)
elif diferencia2 < diferencia :
    if diferencia2 > 2 : 
        print(aprox2, "Hz")
    elif diferencia2 < 2 : 
        print(aprox2, "Hz")
    else : determinarNotaMusical2(aprox2)
else: 
    determinarNotaMusical(aprox)
    determinarNotaMusical2(aprox2)

5582.456
4700.88
d1:  -886.576
d2:  0.0
Es un Re en:  4700.88 Hz
5582.456 Hz


In [50]:
def verificarDiferencia (diferencia, diferencia2, aprox, aprox2):
    resp, resp2 = False, False

    if diferencia < 10 and diferencia > -10:
        resp = True
    else: resp = False
    if diferencia2 < 10 and diferencia2 > -10:
        resp2 = True
    else: resp2 = False

    if resp and resp2 : 
        if diferencia < diferencia2 :
            determinarNotaMusical(aprox)
        elif diferencia > diferencia :
            determinarNotaMusical2(aprox2)
        else: 
            determinarNotaMusical(aprox)
            determinarNotaMusical2(aprox2)
    elif resp: determinarNotaMusical(aprox)
    elif resp2: determinarNotaMusical2(aprox2)
    else : 
        print(aprox, "Hz")
        print(aprox, "Hz")

In [51]:
matplotlib.use('TkAgg')

FRAMES = 1024*8                                   # Tamaño del paquete a procesar
FORMAT = pa.paInt16                               # Formato de lectura INT 16 bits
CHANNELS = 1
Fs = 44100                                        # Frecuencia de muestreo típica para audio

p = pa.PyAudio()

stream = p.open(                                  # Abrimos el canal de audio con los parámeteros de configuración
    format = FORMAT,
    channels = CHANNELS,
    rate = Fs,
    input=True,
    output=True,
    frames_per_buffer=FRAMES
)

# Creamos una gráfica con 2 subplots y configuramos los ejes

fig, (ax,ax1) = plt.subplots(2)

x_audio = np.arange(0,FRAMES,1)
x_fft = np.linspace(0, Fs, FRAMES)
line, = ax.plot(x_audio, np.random.rand(FRAMES),'r')
line_fft, = ax1.semilogx(x_fft, np.random.rand(FRAMES), 'b')

ax.set_ylim(-32500,32500)
ax.ser_xlim = (0,FRAMES)
Fmin = 1
Fmax = 5000
ax1.set_xlim(Fmin,Fmax)

fig.show()

F = (Fs/FRAMES)*np.arange(0,FRAMES//2)                 # Creamos el vector de frecuencia para encontrar la frecuencia dominante

while True:

    data = stream.read(FRAMES)                         # Leemos paquetes de longitud FRAMES
    dataInt = struct.unpack(str(FRAMES) + 'h', data)   # Convertimos los datos que se encuentran empaquetados en bytes

    line.set_ydata(dataInt)                            # Asignamos los datos a la curva de la variación temporal

    M_gk = abs(fourier.fft(dataInt)/FRAMES)            # Calculamos la FFT y la Magnitud de la FFT del paqute de datos

    ax1.set_ylim(0,np.max(M_gk+10)) 
    line_fft.set_ydata(M_gk)                           # Asigmanos la Magnitud de la FFT a la curva del espectro 

    M_gk = M_gk[0:FRAMES//2]                           # Tomamos la mitad del espectro para encontrar la Frecuencia Dominante
    Posm = np.where(M_gk == np.max(M_gk))
    HzMaximo = F[Posm]                                   # Encontramos la frecuencia que corresponde con el máximo de M_gk

    aproximacion = aproximar(FaFaSSolHz, HzMaximo)
    aproximacion2 = aproximar(allHz, HzMaximo)

    diferencia = HzMaximo - aproximacion
    diferencia2 = HzMaximo - aproximacion2

    verificarDiferencia(diferencia, diferencia2, aproximacion, aproximacion2)
    
    fig.canvas.draw()
    fig.canvas.flush_events()

Es un Fa# en:  48.25 Hz
Es un Sol# en:  48.92 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un Fa# en:  48.25 Hz
Es un Sol# en:  48.92 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un Fa# en:  48.25 Hz
Es un Sol# en:  48.92 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un Fa# en:  48.25 Hz
Es un Sol# en:  48.92 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un Fa# en:  48.25 Hz
Es un Sol# en:  48.92 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un La# en:  237.12 Hz
Es un Sol en:  201.0 Hz
Es un Sol# en:  209.68 Hz
Es un

KeyboardInterrupt: 